# Import and Load Libraries

In [1]:
# Python Standard Library and 3rd Party Imports 

import pandas as pd
import numpy as np
import networkx as nx
import scipy
import copy
import re
from collections import Counter, defaultdict
from itertools import chain, combinations
from pathlib import Path

In [2]:
# COBRA imports

import cobra
from cobra import Model, Reaction
from cobra.core import Group, Reaction
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis.fastcc import fastcc
from cobra.io import read_sbml_model, save_matlab_model, write_sbml_model
from cobra.util.solver import linear_reaction_coefficients
from rapidfuzz import fuzz, process
from networkx.algorithms import bipartite
from scipy.linalg import svd
from cobra.util.array import create_stoichiometric_matrix
from collections import deque

# Functions

In [3]:
def are_reactions_interconnected(model, reaction_ids):
    """
    Check whether a list of reactions are all interconnected via shared metabolites.
    
    Parameters:
    - model: cobra.Model
    - reaction_ids: list of reaction IDs to test

    Returns:
    - bool: True if all reactions are connected via shared metabolites
    """
    # Create a graph where nodes = reactions, edges = shared metabolite
    G = nx.Graph()
    G.add_nodes_from(reaction_ids)

    # Build edges based on shared metabolites
    for i, rxn1_id in enumerate(reaction_ids):
        if rxn1_id not in model.reactions:
            continue
        rxn1 = model.reactions.get_by_id(rxn1_id)
        mets1 = set(rxn1.metabolites)

        for rxn2_id in reaction_ids[i+1:]:
            if rxn2_id not in model.reactions:
                continue
            rxn2 = model.reactions.get_by_id(rxn2_id)
            mets2 = set(rxn2.metabolites)

            # Add edge if they share at least one metabolite
            if mets1 & mets2:
                G.add_edge(rxn1_id, rxn2_id)

    # Check if the graph is fully connected
    return nx.is_connected(G)

In [4]:
def get_largest_connected_subgroup(model, reaction_list):
    """
    Identify the largest connected component of a group of reactions based on shared metabolites.

    Parameters:
    - model: cobra.Model
        A COBRA model containing reactions and metabolites.
    - reaction_list: list of str
        A list of reaction IDs to analyze.

    Returns:
    - list of str:
        The subset of reaction IDs from the input that belong to the largest connected component.
        Connectivity is defined by reactions sharing at least one metabolite (bipartite graph: reactions ↔ metabolites).
    """
    G = nx.Graph()
    for rxn_id in reaction_list:
        rxn = model.reactions.get_by_id(rxn_id)
        for met in rxn.metabolites:
            G.add_edge(rxn.id, met.id)

    components = nx.connected_components(G)
    
    reaction_components = [
        set(comp).intersection(reaction_list) for comp in components
    ]
    
    largest = max(reaction_components, key=len, default=[])
    return list(largest)


In [5]:
def lump_reaction(model, coupled_df, reaction_column="Coupled_Reactions", verbose=True, label_type="Group"):
    """
    Create a lumped model by replacing fully coupled reaction groups with pseudo-reactions.

    Parameters:
    - model: cobra.Model
        The original COBRA model (remains unchanged).
    - coupled_df: pd.DataFrame
        DataFrame with a column containing lists of reactions to lump.
    - reaction_column: str
        The column name in `coupled_df` that holds lists of reactions (default: "Coupled_Reactions").
    - verbose: bool
        If True, print progress messages.
    - label_type: str
        Label prefix for naming the pseudo-reactions (e.g., "Group", "Module").

    Returns:
    - model_lumped: cobra.Model
        A copy of the model with pseudo-reactions added and original reactions removed.
    - translation_df: pd.DataFrame
        DataFrame mapping pseudo-reactions to their original reaction IDs.
    """
    import copy
    from cobra import Reaction
    from collections import defaultdict
    import pandas as pd

    model_lumped = copy.deepcopy(model)
    translation_data = []

    for i, row in coupled_df.iterrows():
        group_reactions = row[reaction_column]

        # Filter only reactions that exist in the model
        valid_reactions = [rxn_id for rxn_id in group_reactions if rxn_id in model_lumped.reactions]

        if len(valid_reactions) < 2:
            if verbose:
                print(f"⏭️ Skipping group {i} — fewer than 2 valid reactions.")
            continue

        # Determine common flux bounds
        lower_bounds = [model_lumped.reactions.get_by_id(r).lower_bound for r in valid_reactions]
        upper_bounds = [model_lumped.reactions.get_by_id(r).upper_bound for r in valid_reactions]
        min_lb = max(lower_bounds)
        max_ub = min(upper_bounds)

        # Compute net stoichiometry
        net_stoich = defaultdict(float)
        for rxn_id in valid_reactions:
            rxn = model_lumped.reactions.get_by_id(rxn_id)
            for met, coeff in rxn.metabolites.items():
                net_stoich[met] += coeff
        cleaned_stoich = {met: coeff for met, coeff in net_stoich.items() if abs(coeff) > 1e-10}

        # Create pseudo-reaction
        pseudo_id = f"Pseudo_{label_type}_{i}"
        pseudo_rxn = Reaction(id=pseudo_id)
        pseudo_rxn.name = f"Lumped reaction for: {', '.join(valid_reactions)}"
        pseudo_rxn.lower_bound = min_lb
        pseudo_rxn.upper_bound = max_ub
        pseudo_rxn.add_metabolites(cleaned_stoich)

        # Add and remove reactions
        model_lumped.add_reactions([pseudo_rxn])
        for rxn_id in valid_reactions:
            model_lumped.reactions.remove(model_lumped.reactions.get_by_id(rxn_id))

        # Record mapping
        translation_data.append({
            'Pseudo_Reaction_ID': pseudo_id,
            'Original_Reactions': valid_reactions
        })

        if verbose:
            print(f"✅ {label_type} {i}: Created '{pseudo_id}' from {valid_reactions}")

    translation_df = pd.DataFrame(translation_data)
    return model_lumped, translation_df

In [6]:
def find_precursors(model, biomass_precursors, core_mets, max_depth=5):
    """
    Find core metabolites that can act as precursors to each biomass precursor,
    using reactions in the model. Handles reversibility and stoichiometry.

    Parameters:
        model (cobra.Model): The metabolic model.
        biomass_precursors (list): List of metabolite IDs in the biomass reaction.
        core_mets (list): List of core metabolite IDs.
        max_depth (int): Maximum number of reactions to search backwards.

    Returns:
        dict: {biomass_precursor: set of core_metabolites_that_can_reach_it}
    """
    # Build reverse graph: met_id -> reactions that can produce it
    met_to_producing_rxns = defaultdict(list)

    for rxn in model.reactions:
        for met, coeff in rxn.metabolites.items():
            if coeff > 0:
                # Metabolite is produced in forward direction
                met_to_producing_rxns[met.id].append((rxn, 'forward'))
            elif coeff < 0 and rxn.lower_bound < 0:
                # Metabolite is produced in reverse direction
                met_to_producing_rxns[met.id].append((rxn, 'reverse'))

    result = {}

    for target_met in biomass_precursors:
        visited_mets = set()
        visited_rxns = set()
        queue = deque([(target_met, 0)])  # (metabolite_id, depth)
        precursors = set()

        while queue:
            current_met, depth = queue.popleft()
            if depth >= max_depth:
                continue

            producing_rxns = met_to_producing_rxns.get(current_met, [])

            for rxn, direction in producing_rxns:
                if (rxn.id, direction) in visited_rxns:
                    continue
                visited_rxns.add((rxn.id, direction))

                # Get reactants based on direction
                if direction == 'forward':
                    reactants = rxn.reactants
                else:
                    reactants = rxn.products  # in reverse, products become precursors

                for reactant in reactants:
                    if reactant.id in visited_mets:
                        continue
                    visited_mets.add(reactant.id)

                    if reactant.id in core_mets:
                        precursors.add(reactant.id)
                    else:
                        queue.append((reactant.id, depth + 1))

        result[target_met] = precursors

    return result

In [7]:
def analyze_model(model, model_name, carb_list):
    """
    Run FBA simulations for a COBRA model across multiple carbon source conditions.

    For each carbon source in the provided list, this function:
    - Allows uptake only for that carbon source (by setting its flux bounds).
    - Blocks all other carbon sources.
    - Runs Flux Balance Analysis (FBA).
    - Records which reactions are active (i.e., carry non-zero flux).
    - Stores the model's objective value (e.g., growth rate).

    The function returns:
    1. A summary DataFrame with objective values for each carbon source.
    2. A full flux activity matrix (True/False for each reaction in each condition).
    3. A filtered version of the matrix showing only reactions that are active in some but not all conditions.

    Parameters
    ----------
    model : cobra.Model
        The COBRA model to simulate.

    model_name : str
        Identifier for the model (used in the output DataFrame).

    carb_list : list of str
        List of exchange reaction IDs corresponding to carbon sources to be tested.

    Returns
    -------
    results_df : pandas.DataFrame
        DataFrame with columns ['Model', 'Carbon_Source', 'Objective_Value'].

    flux_activity_df : pandas.DataFrame
        Boolean DataFrame with reactions as rows and carbon sources as columns.
        True indicates the reaction carried flux in that condition.

    filtered_flux_activity_df : pandas.DataFrame
        Subset of `flux_activity_df` containing only reactions that are active
        in some but not all carbon source conditions.
    """
    flux_threshold = 1e-6  # Minimum threshold to consider flux as active

    results = []
    flux_activity_df = pd.DataFrame({'Reaction': [rxn.id for rxn in model.reactions]})
    flux_activity_df.set_index('Reaction', inplace=True)

    for carb in carb_list:
        model_temp = model.copy()

        # Block all carbon sources except the current one
        for rxn_id in carb_list:
            if rxn_id in model_temp.reactions:
                rxn = model_temp.reactions.get_by_id(rxn_id)
                if rxn_id == carb:
                    rxn.lower_bound = -10.0
                    rxn.upper_bound = 1000.0
                else:
                    rxn.lower_bound = 0.0
                    rxn.upper_bound = 0.0

        # Run FBA
        solution = model_temp.optimize()

        # Record objective value
        results.append({
            'Model': model_name,
            'Carbon_Source': carb,
            'Objective_Value': solution.objective_value
        })

        # Record which reactions carry flux
        active_flux = solution.fluxes.abs() > flux_threshold
        flux_activity_df[carb] = active_flux

    # Filter reactions active in some but not all conditions
    filtered_flux_activity_df = flux_activity_df[
        ~(flux_activity_df.all(axis=1) | ~flux_activity_df.any(axis=1))
    ]

    results_df = pd.DataFrame(results)
    return results_df, flux_activity_df, filtered_flux_activity_df


# Import Data

In [8]:
# Set the project root path by going up from the notebook location
notebook_dir = Path(__file__).parent if '__file__' in globals() else Path().resolve()
project_root = notebook_dir.parent.parent  # Go up from /code/GEM_Reduction/

# Construct raw paths 
raw_data_path = project_root / "data" / "raw" 
raw_sbml_path = raw_data_path / "sbml_files" 
raw_mat_path = raw_data_path / "matlab_files" 
raw_csv_path = raw_data_path / "csv_files" 

# Construct processed paths 
processed_data_path = project_root / "data" / "processed" 
processed_sbml_path = processed_data_path / "sbml_files" 
processed_csv_path = processed_data_path / "csv_files" 

In [10]:
# read model
model = read_sbml_model(raw_sbml_path / "iAF1260.xml")
subsystem_df = pd.read_csv(raw_csv_path / 'iAF1260_subsystem_assignments.csv')

In [11]:
# Load a curated list of exchange reactions from file
exchange_df_full = pd.read_csv(raw_csv_path / 'iAF1260_exchanges.csv')
final_exchanges_df = pd.read_csv(raw_csv_path / 'iAF1260_context_exchanges.csv')

In [19]:
# import pairs coupled and reactions blocked by F2C2
Coupled_Pairs_df = pd.read_csv(raw_csv_path / 'fctable_iAF1260_context.csv', header=None)
F2C2_Blocked_Reactions_df = pd.read_csv(raw_csv_path / 'blocked_reactions_iAF1260_context.csv', header=None)

### Basic model investigation

Model Basics
- Model ID: iAF1260
- Number of Reactions: 2382
- Number of Metabolites: 1668
- Number of Genes: 1261
- Number of Exchange Reactions: 299
- Reversible Reactions: 575
- Irreversible Reactions: 1807
- Objective reaction(s): R_BIOMASS_Ec_iAF1260_core_59p81M

- FBA Objective value (biomass): 0.7367

In [20]:
# Get the stoichiometric matrix as a NumPy array
stoich_dense = create_stoichiometric_matrix(model)  # Already a NumPy array

# Compute the rank of the stoichiometric matrix
rank = np.linalg.matrix_rank(stoich_dense)

# Degrees of freedom
num_reactions = len(model.reactions)
dof = num_reactions - rank

# Output
print(f"Stoichiometric Matrix Rank: {rank}")
print(f"Number of Reactions: {num_reactions}")
print(f"Degrees of Freedom: {dof}")

Stoichiometric Matrix Rank: 1630
Number of Reactions: 2382
Degrees of Freedom: 752


# Model Exchanges

In [21]:
 # Create a deep copy of the model to avoid modifying the original
model_copy = copy.deepcopy(model)

**Step 1**: Create Files to invest exchanges

In [22]:
# Step 1: Run FBA on the model
solution = model_copy.optimize()

# Step 2: Extract all exchange reactions from the model
exchange_reactions = model_copy.exchanges

# Step 3: Create a DataFrame summarizing each exchange reaction
exchange_df = pd.DataFrame({
    "ID": [rxn.id for rxn in exchange_reactions],
    "Name": [rxn.name for rxn in exchange_reactions],
    "Lower_FB": [rxn.lower_bound for rxn in exchange_reactions],
    "Upper_FB": [rxn.upper_bound for rxn in exchange_reactions],
})

# Step 4: Add the FBA flux value for each exchange reaction
exchange_df["Flux"] = exchange_df["ID"].apply(lambda rxn_id: solution.fluxes.get(rxn_id, 0.0))

# Step 5: Identify which reactions are active (i.e., flux is significantly non-zero)
exchange_df["Active_in_FBA"] = exchange_df["Flux"].apply(lambda v: abs(v) > 1e-6)

# (Optional) Step 6: Duplicate 'Active_in_FBA' column (may be removed if not needed)
exchange_df["Active_in_FBA_Copy"] = exchange_df["Active_in_FBA"]

# Step 7: Filter to only active exchange reactions and create a copy
exchange_df_sub = exchange_df[exchange_df["Active_in_FBA_Copy"] == True].copy()

# Step 8: Add 'Import' column:
#   - True if flux < 0 (uptake)
#   - False if flux > 0 (secretion)
#   - NaN if zero flux (shouldn't appear here due to filter)
exchange_df_sub["Import"] = exchange_df_sub["Flux"].apply(
    lambda v: True if v < 0 else (False if v > 0 else pd.NA)
)

# Step 10: Filter the list to those reactions marked for inclusion
exchange_df2 = exchange_df_full[exchange_df_full["Include"] == True]

# Step 11: Merge the curated exchange list (exchange_df2) with activity/Import info from FBA
# Left join ensures all included reactions are preserved; Import info added where available
merged_df = exchange_df2.merge(
    exchange_df_sub[["ID", "Import"]],
    on="ID",
    how="left"
)

**Step 2**: Import modified exchange reactions incl directionalities

In [23]:
 # Create a deep copy of the model to avoid modifying the original
model_copy2 = copy.deepcopy(model)

In [24]:
# Get IDs from final_exchanges_df
allowed_ids = set(final_exchanges_df["ID"])

# Step 1: Block all exchange reactions NOT in the list
for rxn in model_copy2.exchanges:
    if rxn.id not in allowed_ids:
        rxn.lower_bound = 0.0
        rxn.upper_bound = 0.0

# Step 2 and 3: Handle Import = False and Import = True separately
for _, row in final_exchanges_df.iterrows():
    rxn_id = row["ID"]
    is_import = row["Import"]

    # Get the reaction from the model
    rxn = model_copy2.reactions.get_by_id(rxn_id)

    if is_import is False:
        # Export reaction: restrict lower bound to 0, keep upper as-is
        rxn.lower_bound = 0.0

    elif is_import is True:
        # Import reaction: reverse it and adjust bounds

        # Store previous lower bound before changing it
        prev_lower = rxn.lower_bound

        # Create reversed reaction
        reversed_rxn = Reaction(id=f"{rxn.id}_reversed")
        reversed_rxn.name = f"Reversed {rxn.name}"

        # Reverse the stoichiometry (reactants <-> products)
        reversed_rxn.add_metabolites({met: -coef for met, coef in rxn.metabolites.items()})

        # Set bounds: lower = 0, upper = abs(previous lower)
        reversed_rxn.lower_bound = 0.0
        reversed_rxn.upper_bound = abs(prev_lower)

        # Add the new reaction to the model and remove the old one
        model_copy2.add_reactions([reversed_rxn])
        model_copy2.remove_reactions([rxn])

In [25]:
# List of carbon source exchange reactions
carb_list = ['EX_glc__D_e_reversed', 'EX_fru_e_reversed', 'EX_sucr_e_reversed', 'EX_arab__L_e_reversed', 'EX_mnl_e_reversed']

# Set bounds for each reaction
for rxn_id in carb_list:
    if rxn_id in model_copy2.reactions:
        rxn = model_copy2.reactions.get_by_id(rxn_id)
        rxn.lower_bound = 0.0
        rxn.upper_bound = 2.0
    else:
        print(f"Reaction {rxn_id} not found in the model.")

In [26]:
### Export as .mat for matlab applications

save_matlab_model(model_copy2, raw_mat_path / "iAF1260_context.mat") # use generic model to get unbiased couplings (filter later!)

**Step 2**: Run FASTCC

In [27]:
# Reduce model with COBRApy FASTCC 
consistent_generic_model = fastcc(model_copy2)

In [28]:
# Get the stoichiometric matrix as a NumPy array
stoich_dense = create_stoichiometric_matrix(consistent_generic_model)  # Already a NumPy array

# Compute the rank of the stoichiometric matrix
rank = np.linalg.matrix_rank(stoich_dense)

# Degrees of freedom
num_reactions = len(consistent_generic_model.reactions)
dof = num_reactions - rank

# Output
print(f"Stoichiometric Matrix Rank: {rank}")
print(f"Number of Reactions: {num_reactions}")
print(f"Degrees of Freedom: {dof}")

Stoichiometric Matrix Rank: 861
Number of Reactions: 1313
Degrees of Freedom: 452


**Step 2.1**: Make F2C2 Data accessible

In [29]:
# Add reaction annotations to match F2C2 blocked reactions
model_rxns = [rxn.id for rxn in model_copy2.reactions]
F2C2_Blocked_Reactions_df.loc[len(F2C2_Blocked_Reactions_df)] = model_rxns

# Identify unblocked reactions from the second row (index 1)
unblocked_mask = F2C2_Blocked_Reactions_df.loc[0] == 0
unblocked_reactions = F2C2_Blocked_Reactions_df.loc[1][unblocked_mask].tolist()

# Update Coupled_Pairs_df index and columns with unblocked reactions
Coupled_Pairs_df.index = unblocked_reactions
Coupled_Pairs_df.columns = unblocked_reactions

In [30]:
# Filter dataframe for reactions in model
# Step 1: Get all reaction IDs from your COBRApy model
model_rxns = set([rxn.id for rxn in consistent_generic_model.reactions])

# Step 2: Identify which rows/columns are in the model
valid_rxns = Coupled_Pairs_df.index.intersection(model_rxns)

# Step 3: Subset the DataFrame to keep only those reactions
Coupled_Pairs_df = Coupled_Pairs_df.loc[valid_rxns, valid_rxns]

In [31]:
# Get number of how many fully coupled pairs
fully_coupled_count = int(((Coupled_Pairs_df == 1).sum().sum()) - len(Coupled_Pairs_df))

*SideQuest* : Investigate difference in 'blocking' between FASTCC and F2C2

In [32]:
# Extract and convert reactions from models to sets for fast comparison
generic_rxns = {rxn.id for rxn in consistent_generic_model.reactions}
f2c2_unblocked_rxns = set(unblocked_reactions)

# Compare overlaps and differences
overlap_generic = list(generic_rxns & f2c2_unblocked_rxns)

# Reactions unblocked by F2C2 but removed by FASTCC
missing_from_generic = list(f2c2_unblocked_rxns - generic_rxns)

# Reactions kept by FASTCC but blocked by F2C2 (ideally empty)
unexpected_in_generic = list(generic_rxns - f2c2_unblocked_rxns)

**Step 2**: Create 'Fully Coupled Clusters'

In [33]:
# Build a graph of fully coupled reactions
G = nx.Graph()

for row in Coupled_Pairs_df.index:
    for col in Coupled_Pairs_df.columns:
        if Coupled_Pairs_df.loc[row, col] == 1 and row != col:
            G.add_edge(row, col)

# Find connected components (fully coupled groups)
coupled_groups = list(nx.connected_components(G))

In [34]:
# Build reaction groups with net stoichiometry and input/output metabolites
combined_data = []
for group in coupled_groups:
    group_reactions = list(group)

    # Compute net stoichiometry
    stoich = defaultdict(float)
    for rxn_id in group_reactions:
        rxn = consistent_generic_model.reactions.get_by_id(rxn_id)
        for met, coeff in rxn.metabolites.items():
            stoich[met] += coeff

    inputs = [met.id for met, coeff in stoich.items() if coeff < 0]
    outputs = [met.id for met, coeff in stoich.items() if coeff > 0]

    combined_data.append({
        "Coupled_Reactions": group_reactions,
        "Num_Inputs": len(inputs),
        "Num_Outputs": len(outputs),
        "Input_Metabolites": inputs,
        "Output_Metabolites": outputs
    })

# Create initial DataFrame
Fully_Coupled_df = pd.DataFrame(combined_data)

In [35]:
# Flatten reaction lists to count how many should be removed
all_reactions = Fully_Coupled_df['Coupled_Reactions'].explode()

ex_removed_count = all_reactions.str.startswith('EX_').sum()
biomass_removed_flag = 'BIOMASS_Ec_iAF1260_core_59p81M' in set(all_reactions)
missing_set = set(missing_from_generic)
missing_removed_count = all_reactions.isin(missing_set).sum()

# Clean the reaction lists: remove EX_, biomass, and missing reactions
Fully_Coupled_df['Coupled_Reactions'] = Fully_Coupled_df['Coupled_Reactions'].apply(
    lambda rxns: [
        r for r in rxns
        if not r.startswith('EX_')
        and r != 'BIOMASS_Ec_iAF1260_core_59p81M'
        and r not in missing_set
    ]
)

# Drop empty groups
Fully_Coupled_df = Fully_Coupled_df[Fully_Coupled_df['Coupled_Reactions'].str.len() > 0]

# Report removals
print(f"Total 'EX_' reactions removed: {ex_removed_count}")
print(f"'BIOMASS_Ec_iAF1260_core_59p81M' was removed: {biomass_removed_flag}")
print(f"Total reactions removed due to fastcc removal: {missing_removed_count}")

Total 'EX_' reactions removed: 25
'BIOMASS_Ec_iAF1260_core_59p81M' was removed: True
Total reactions removed due to fastcc removal: 0


In [36]:
# Check cluster connectivity
Fully_Coupled_df['Cluster'] = Fully_Coupled_df['Coupled_Reactions'].apply(
    lambda rxn_list: are_reactions_interconnected(consistent_generic_model, rxn_list)
)

# Count number of reactions
Fully_Coupled_df['Num_Reactions'] = Fully_Coupled_df['Coupled_Reactions'].apply(len)

# Remove disconnected pairs of size 2
Fully_Coupled_df = Fully_Coupled_df[~((Fully_Coupled_df['Num_Reactions'] == 2) & (Fully_Coupled_df['Cluster'] == False))]

# Fix broken clusters with more than 2 reactions
mask = (Fully_Coupled_df['Cluster'] == False) & (Fully_Coupled_df['Num_Reactions'] > 2)
Fully_Coupled_df.loc[mask, 'Cleaned_Coupled_Reactions'] = Fully_Coupled_df.loc[mask, 'Coupled_Reactions'].apply(
    lambda rxns: get_largest_connected_subgroup(consistent_generic_model, rxns)
)

# Keep reactions as-is for connected groups
Fully_Coupled_df['Cleaned_Coupled_Reactions'] = Fully_Coupled_df['Cleaned_Coupled_Reactions'].fillna(Fully_Coupled_df['Coupled_Reactions'])

# Track removed reactions per group
Fully_Coupled_df['Removed_Reactions'] = Fully_Coupled_df.apply(
    lambda row: list(set(row['Coupled_Reactions']) - set(row['Cleaned_Coupled_Reactions'])),
    axis=1
)

In [37]:
# Prepare final DataFrame for analysis
selected_columns = ['Cleaned_Coupled_Reactions', 'Removed_Reactions']
Lumping_Couples_df = Fully_Coupled_df[selected_columns].copy()

# Drop single-reaction groups
Lumping_Couples_df = Lumping_Couples_df[Lumping_Couples_df['Cleaned_Coupled_Reactions'].str.len() > 1]

# Map reactions to subsystems
subsystem_map = dict(zip(subsystem_df['iAF1260_BIGG'], subsystem_df['Subsystem']))
Lumping_Couples_df['Subsystem'] = Lumping_Couples_df['Cleaned_Coupled_Reactions'].apply(
    lambda rxn_list: list(set(subsystem_map.get(rxn, 'Unknown') for rxn in rxn_list))
)

# Add summary columns
Lumping_Couples_df['Num_Subsystems'] = Lumping_Couples_df['Subsystem'].apply(len)
Lumping_Couples_df['Num_Reactions'] = Lumping_Couples_df['Cleaned_Coupled_Reactions'].apply(len)

**Step 3**: Lump fully coupled clusters into pseudo-reactions

In [38]:
model_lumped, translation_df = lump_reaction(consistent_generic_model, Lumping_Couples_df, reaction_column="Cleaned_Coupled_Reactions",verbose=False)

In [39]:
# Identify and remove orphaned metabolites used only in pseudo-reactions
metabolites_to_remove = []

for met in model_lumped.metabolites:
    # Get all reactions involving this metabolite
    associated_rxns = [rxn.id for rxn in met.reactions]
    
    # If the metabolite only appears in one reaction AND it's a pseudo-reaction
    if len(associated_rxns) == 1 and associated_rxns[0].startswith("Pseudo_"):
        metabolites_to_remove.append(met)

# Remove them from the model
model_lumped.remove_metabolites(metabolites_to_remove)

print(f"🧹 Removed {len(metabolites_to_remove)} internal metabolites used only in pseudo-reactions.")

🧹 Removed 0 internal metabolites used only in pseudo-reactions.


In [40]:
# Get the stoichiometric matrix as a NumPy array
stoich_dense = create_stoichiometric_matrix(model_lumped)  

# Compute the rank of the stoichiometric matrix
rank = np.linalg.matrix_rank(stoich_dense)

# Degrees of freedom
num_reactions = len(model_lumped.reactions)
dof = num_reactions - rank

# Output
print(f"Stoichiometric Matrix Rank: {rank}")
print(f"Number of Reactions: {num_reactions}")
print(f"Degrees of Freedom: {dof}")

Stoichiometric Matrix Rank: 515
Number of Reactions: 946
Degrees of Freedom: 431


**Step 4**: Sanity Check

In [41]:
sol_orig = consistent_generic_model.optimize()
sol_lumped = model_lumped.optimize()

print(f"Original model FBA objective: {sol_orig.objective_value:.6f}")
print(f"Lumped model FBA objective:   {sol_lumped.objective_value:.6f}")

if abs(sol_orig.objective_value - sol_lumped.objective_value) < 1e-6:
    print("✅ FBA optimum preserved.")
else:
    print("⚠️ FBA optimum changed — check lumping effects.")

Original model FBA objective: 0.951942
Lumped model FBA objective:   0.951942
✅ FBA optimum preserved.


In [42]:
# Get only the pseudo-reactions
pseudo_rxns = [rxn.id for rxn in model_lumped.reactions if rxn.id.startswith("Pseudo_")]

# Run FVA on those reactions
fva_result = flux_variability_analysis(model_lumped, reaction_list=pseudo_rxns, fraction_of_optimum=0.0)

# Check which pseudo-reactions are blocked (min=max=0)
blocked = fva_result[(fva_result["minimum"] == 0) & (fva_result["maximum"] == 0)]

print(f"Total pseudo-reactions: {len(pseudo_rxns)}")
print(f"Blocked pseudo-reactions: {len(blocked)}")

if not blocked.empty:
    print("⚠️ Some pseudo-reactions are blocked:")
    print(blocked)
else:
    print("✅ All pseudo-reactions are feasible.")

Total pseudo-reactions: 194
Blocked pseudo-reactions: 0
✅ All pseudo-reactions are feasible.


# Subsystem based lumping

**Step 1**: Group by module

In [43]:
# Dictionary with subsystem mapping
subsystem_map = dict(zip(subsystem_df['iAF1260_BIGG'], subsystem_df['Subsystem']))

# Create working copy
translation_cp_df = translation_df.copy()

# Apply the logic inline, returning a tuple (Subsystem, Unambiguous_Subsystem), then split into columns
translation_cp_df[['Subsystem', 'Unambiguous_Subsystem']] = translation_cp_df['Original_Reactions'].apply(
    lambda reactions: (
        (lambda subsystems: (
            Counter(subsystems).most_common(1)[0][0] if subsystems else None,
            True if len(set(Counter(subsystems).values())) == 1 and len(set(subsystems)) == 1 else False
        ))([subsystem_map[rxn] for rxn in reactions if rxn in subsystem_map])
    )
).apply(pd.Series)

# Create new rows from subsystem_df
new_rows = pd.DataFrame({
    'Pseudo_Reaction_ID': subsystem_df['iAF1260_BIGG'],
    'Original_Reactions': [None] * len(subsystem_df),
    'Subsystem': subsystem_df['Subsystem'],
    'Unambiguous_Subsystem': [True] * len(subsystem_df)  # Set to True explicitly
})

# Append new rows to the main DataFrame
translation_cp_df = pd.concat([translation_cp_df, new_rows], ignore_index=True)

In [44]:
#  Extract all reaction IDs from model_lumped
model_reaction_ids = [rxn.id for rxn in model_lumped.reactions]

# Subset translation_df to only those rows with matching Pseudo_Reaction_ID
subset_df = translation_cp_df[translation_cp_df['Pseudo_Reaction_ID'].isin(model_reaction_ids)].copy()

In [45]:
# Group by 'Subsystem' and aggregate
grouped_df = subset_df.groupby('Subsystem').agg(
    Members=('Pseudo_Reaction_ID', list),
    Member_Count=('Pseudo_Reaction_ID', 'count')
).reset_index()

**Step 2**: Remove modules that cannot be lumped

Note: For Core, reference: *Ataman, et al. "redGEM: Systematic reduction ... consistent core metabolic models." PLoS computational biology 13.7 (2017): e1005444.*

In [46]:
Forbidden_subsystems_list = ['S_Exchange','S_Unassigned','S_']

Core_subsystems_list = ['S_GlycolysisGluconeogenesis','S_Pentose_Phosphate_Pathway','S_Citric_Acid_Cycle','S_Glyoxylate_Metabolism','S_Pyruvate_Metabolism','S_Oxidative_Phosphorylation']

Remove_list = Forbidden_subsystems_list + Core_subsystems_list

In [47]:
# Subset: rows to remove (core)
Core_grouped_df = grouped_df[grouped_df['Subsystem'].isin(Remove_list)].copy()

# Subset: rows to keep (non-core)
Noncore_grouped_df = grouped_df[~grouped_df['Subsystem'].isin(Remove_list)].copy()

**Step 3**: Connect to relevance for biomass reaction

In [48]:
# Get metabolites required by the biomass reaction (educts)
biomass_rxn = model_lumped.reactions.get_by_id("BIOMASS_Ec_iAF1260_core_59p81M")
biomass_educts = {met.id for met, coeff in biomass_rxn.metabolites.items() if coeff < 0}

# Containers for new columns
educts_list = []
products_list = []
internal_list = []
biomass_outputs = []
biomass_flags = []

# Iterate over each subsystem group (each row)
for _, row in Noncore_grouped_df.iterrows():
    reaction_ids = row['Members']
    stoich = defaultdict(float)

    # Accumulate net stoichiometry across all reactions
    for rxn_id in reaction_ids:
        if rxn_id not in model_lumped.reactions:
            continue
        rxn = model_lumped.reactions.get_by_id(rxn_id)
        for met, coeff in rxn.metabolites.items():
            stoich[met.id] += coeff

    # Classify metabolites
    educt_met = [m for m, c in stoich.items() if c < 0]
    product_met = [m for m, c in stoich.items() if c > 0]

    # Biomass-relevant products
    biomass_met = [m for m in product_met if m in biomass_educts]
    biomass_relevant = len(biomass_met) > 0

    # Append results
    educts_list.append(educt_met)
    products_list.append(product_met)
    biomass_outputs.append(biomass_met)
    biomass_flags.append(biomass_relevant)

# Assign new columns to grouped_df
Noncore_grouped_df['educt_met'] = educts_list
Noncore_grouped_df['product_met'] = products_list
Noncore_grouped_df['biomass'] = biomass_outputs
Noncore_grouped_df['biomass_relevant'] = biomass_flags

Noncore_grouped_df['Interconnected'] = Noncore_grouped_df['Members'].apply(
    lambda rxns: are_reactions_interconnected(model_lumped, rxns)
)

Noncore_grouped_df.sort_values(by='Member_Count',ascending=False,inplace=True)

In [49]:
# Find largest subclusters that are interconnected for non-interconnected groups
mask = (Noncore_grouped_df['Interconnected'] == False)

Noncore_grouped_df.loc[mask, 'Members_Cleaned_Reactions'] = Noncore_grouped_df.loc[mask, 'Members'].apply(
    lambda rxns: get_largest_connected_subgroup(model_lumped, rxns)
)

In [50]:
Noncore_grouped_df['Members_Cleaned_Reactions_Number'] = Noncore_grouped_df['Members_Cleaned_Reactions'].apply(lambda x: len(x) if isinstance(x, list) else 0)

In [51]:
Noncore_grouped_df["Leftover_Members"] = Noncore_grouped_df.apply(
    lambda row: list(set(row["Members"]) - set(row["Members_Cleaned_Reactions"]))
    if isinstance(row["Members_Cleaned_Reactions"], list) else np.nan,
    axis=1
)

In [52]:
# Find second largest subclusters that are interconnected for non-interconnected groups

mask = Noncore_grouped_df["Members_Cleaned_Reactions"].notna()

# Apply function only to rows where the mask is True
Noncore_grouped_df.loc[mask, 'Leftover_Cleaned_Members'] = Noncore_grouped_df.loc[mask, 'Leftover_Members'].apply(
    lambda rxns: get_largest_connected_subgroup(model_lumped, rxns)
)

In [53]:
Noncore_grouped_df['Leftover_Cleaned_Members_Number'] = Noncore_grouped_df['Leftover_Cleaned_Members'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Lump Subsystems

**Step 1**: Truly interconnected subsystems

In [54]:
# subset to initially true ones
copy1_df = Noncore_grouped_df[Noncore_grouped_df['Interconnected'] == True]

In [55]:
copy1, translation_df = lump_reaction(model_lumped, copy1_df, reaction_column="Members",verbose=False,label_type="Subsystem")

In [56]:
# Get the stoichiometric matrix as a NumPy array
stoich_dense = create_stoichiometric_matrix(copy1)  

# Compute the rank of the stoichiometric matrix
rank = np.linalg.matrix_rank(stoich_dense)

# Degrees of freedom
num_reactions = len(copy1.reactions)
dof = num_reactions - rank

# Output
print(f"Stoichiometric Matrix Rank: {rank}")
print(f"Number of Reactions: {num_reactions}")
print(f"Degrees of Freedom: {dof}")

Stoichiometric Matrix Rank: 364
Number of Reactions: 575
Degrees of Freedom: 211


**Step 2**: Largest Subclusters of non interconnected Subsystems

In [57]:
copy2_df = Noncore_grouped_df[Noncore_grouped_df["Members_Cleaned_Reactions"].notna()]

In [58]:
copy2, translation_df = lump_reaction(copy1, copy2_df, reaction_column="Members_Cleaned_Reactions",verbose=False,label_type="Subsystem_Cluster1")

In [59]:
# Get the stoichiometric matrix as a NumPy array
stoich_dense = create_stoichiometric_matrix(copy2)  

# Compute the rank of the stoichiometric matrix
rank = np.linalg.matrix_rank(stoich_dense)

# Degrees of freedom
num_reactions = len(copy2.reactions)
dof = num_reactions - rank

# Output
print(f"Stoichiometric Matrix Rank: {rank}")
print(f"Number of Reactions: {num_reactions}")
print(f"Degrees of Freedom: {dof}")

Stoichiometric Matrix Rank: 140
Number of Reactions: 171
Degrees of Freedom: 31


# Validation

In [61]:
# Run analysis for both models
results_copy2, flux_activity_copy2, filtered_flux_copy2 = analyze_model(copy2, 'copy2',carb_list)
results_model_copy2, flux_activity_model_copy2, filtered_flux_model_copy2 = analyze_model(model_copy2, 'model_copy2',carb_list)

In [62]:
Objective_comparison_df = pd.concat([results_copy2, results_model_copy2], axis=0)

In [63]:
Objective_comparison_df.to_csv(raw_csv_path / 'iAF1260_context_FBA_results.csv')